In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import sys
    # caution: path[0] is reserved for script path (or '' in REPL)

In [7]:
class Node :
    def __init__(self , feat_idx = None, threshold = None , left_child = None , right_child = None , * , value = None) :
        self.feat_idx = feat_idx
        self.threshold = threshold
        self.left_child = left_child
        self.right_child = right_child
        self.value = value
    def is_leaf_node (self) :
        return self.value is not None
def entropy (y) :
    return -np.sum (np.unique (y , return_counts= True)[1]/len (y) * np.log2(np.unique (y , return_counts= True)[1]/len (y)) )
def Entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

class decision_tree :
    def __init__(self , max_depth = 100 , min_sample_split = 3, n_features = 10 , ) :
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.root = None
        self.n_features = 5
        
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._grow_tree(X, y)
        
    def most_common_label (self, Y) :
        return Counter (Y).most_common(1)[0][0]
        
    def _grow_tree (self , X , y , depth = 0 ):
        m , n = X.shape
        n_labels = len (np.unique(y))
        if (depth >= self.max_depth or m < self.min_sample_split or n_labels == 1) :
            leaf_value = self.most_common_label(y)
            return Node (value= leaf_value)
        else :
            feature_idx = np.random.choice (n , self.n_features , replace = False)
            best_feature , best_threshold = self._best_criteria(X , y , feature_idx)
            
            
            left_idx , right_idx = self._split (X[:,best_feature] , best_threshold)
            left_child = self._grow_tree ( X[left_idx , :] , y[left_idx] , depth +1) 
            right_child = self._grow_tree ( X[right_idx, :] , y[right_idx] , depth +1)
            return Node (best_feature , best_threshold ,left_child , right_child)
            
    def _best_criteria(self, X, y , feature_idx) :
        best_ig = -1 
        best_feature_idx , best_threshold = None , None
        for idx in feature_idx :
            X_col_idx = X[: , idx]
            thresholds = np.unique(X_col_idx)
            for threshold in thresholds :
                ig = self._information_gain (y , X_col_idx  , threshold)
                if (ig > best_ig) :
                    best_ig = ig
                    best_feature_idx = idx
                    best_threshold = threshold
        return best_feature_idx , best_threshold
    
    
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs
    
    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    
    def predict (self ,X) :
        result =np.array ( [self._traverse_tree (x , self.root) for x in X ] )
        return result
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feat_idx] <= node.threshold:
            return self._traverse_tree(x, node.left_child)
        return self._traverse_tree(x, node.right_child)

In [79]:
class Random_Forest (decision_tree) :
    def __init__(self, max_depth = 100 , min_sample_split = 3, n_features = 10 ,number_of_tree =100):
        super().__init__(max_depth , min_sample_split , n_features  )
        self.number_of_tree = number_of_tree
    def bootstrap (self , X , y) :
        m , n = X.shape
        n_features = int (np.round(np.log2(n)))
        random_feature = np.random.choice ( n ,n_features , replace =False)
        random_bootstrap_sample = np.random.choice (m , size= m , replace = True) 
        return X[random_bootstrap_sample], y[random_bootstrap_sample]
    def fit (self , X , y ) :
        self.trees = []
        for _ in range(self.number_of_tree) :
            dec_tree = decision_tree ()
            random_X , random_y= self.bootstrap (X , y)
            dec_tree.fit (random_X , random_y)
            self.trees.append(dec_tree)
    def predict (self , X) :
        preds = np.array([tree.predict(X) for tree in self.trees])
        preds = np.swapaxes (preds , 0 , 1)
        preds = np.array ([Counter (pred).most_common(1)[0][0] for pred in preds])
        return preds
            

def accuracy (predict , test) :
        return np.sum( predict == test)/ len(predict)

In [81]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
dataset = datasets.load_breast_cancer ()
X = dataset.data
y = dataset.target
X_train , X_test , Y_train , Y_test = train_test_split ( X , y , test_size= 0.2 , random_state= 42 )

clf = Random_Forest (number_of_tree= 500)
clf.fit (X_train , Y_train)
y_pred = clf.predict (X_test)
accuracy (y_pred , Y_test)

0.42105263157894735

In [68]:
a = np.array ([[21, 22 ,23],
 [11, 22, 33],
 [43 ,77 ,89]])
a[np.array([0,1])][:,np.array([0,1])]

array([[21, 22],
       [11, 22]])